In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ev = pd.read_csv('event.csv')
df_uv = pd.read_csv('user_page_view.csv')

In [3]:
df_pg = pd.read_csv('page_topic.csv')

In [3]:
df_ev.head()

,displayId,timestamp,pageId,widgetId,userId,device,OS,browser
0,4706262,1578429005696,3543873,6262,2688642,0,0,0
1,4706267,1578429007726,6245475,607,2688641,1,3,0
2,4706260,1578429012060,4416499,11458,2688638,0,0,1
3,4706255,1578429017218,6246028,9358,1962852,0,0,0
4,4706256,1578429021388,5327047,9358,2687719,0,0,0


In [4]:
df_uv.head()

,userId,pageId,timestamp
0,821961,8116,1579599211445
1,15321,9533442,1579599211443
2,1125090,9410379,1579599211440
3,407101,8616213,1579599211429
4,781615,9543366,1579599211429


In [9]:
df_uv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67229821 entries, 0 to 67229820
Data columns (total 3 columns):
userId       int64
pageId       int64
timestamp    int64
dtypes: int64(3)
memory usage: 1.5 GB


In [64]:
df_pg['pageId'].nunique()

7025818

In [55]:
df_uv['pageId'].nunique()

1265521

In [56]:
df_ev['pageId'].nunique()

188417

In [4]:
df_uvg = df_uv.groupby(['userId','pageId']).agg({'timestamp':'count'}).reset_index()

In [42]:
df_evg = df_ev.groupby(['userId','pageId']).agg({'timestamp':'count'}).reset_index()

In [57]:
df_evg.rename(columns={'timestamp': 'click_number'}, inplace=True)
df_evg.sample(10)

,userId,pageId,click_number
409842,966002,9236347,1
285546,653019,9742835,1
937645,2619089,6293869,1
653449,1730421,7776612,1
337358,777770,9575312,1
811749,2213276,5946,1
562496,1445179,8249519,1
492968,1214949,6591079,1
807092,2192533,4141373,1
476938,1172174,8927437,1


In [5]:
df_uvg.rename(columns={'timestamp': 'visit_number'}, inplace=True)
df_uvg.sample(10)

,userId,pageId,visit_number
5248829,284912,7040782,2
20142040,1450156,7549561,1
27004024,2198964,8708006,1
10163907,616425,6568922,2
19755961,1406204,8240599,2
4535667,235253,7951327,9
5361403,292834,7361406,1
6289239,348472,7729197,1
1440036,79727,7470217,1
19295654,1346962,6990014,3


In [51]:
df_uvg['visit_number'].max()

15287

In [58]:
df_new = pd.merge(df_evg, df_uvg, how = 'inner', on = ['pageId','userId'])

In [40]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 835634 entries, 0 to 835633
Data columns (total 9 columns):
displayId       835634 non-null int64
timestamp       835634 non-null int64
pageId          835634 non-null int64
widgetId        835634 non-null int64
userId          835634 non-null int64
device          835634 non-null int64
OS              835634 non-null int64
browser         835634 non-null int64
visit_number    835634 non-null int64
dtypes: int64(9)
memory usage: 63.8 MB


In [60]:
df_new.sample(10)

,userId,pageId,click_number,visit_number
293792,1292744,7364449,2,5
154609,865298,9000588,1,1
223743,1077655,8584670,1,1
449493,1796480,6996524,1,1
23509,140787,6376615,1,2
506307,1964083,21048,1,2
303918,1319126,7748115,1,2
569884,2143559,4094218,1,2
640101,2368966,31814,1,1
58294,366370,8533409,1,2


In [6]:
df_new_2 = pd.merge(df_uvg, df_pg, how = 'inner', on = 'pageId')
df_new_2.sample(10)

,userId,pageId,visit_number,topicId,confidence
14466058,1019718,197,6,37,0.022959
31348308,297563,8377233,1,10,0.558779
84460340,611734,6303202,1,40,0.025093
32043117,1084807,7922541,1,24,0.036046
24556551,34399,7951112,1,7,0.441412
20082935,574426,6278960,1,12,0.010374
12915073,1049395,7966673,6,1,0.043527
67283378,1862030,57556,1,44,0.120335
59524077,197699,5733482,3,17,0.073100
82568625,631392,8245135,1,7,0.486227


In [7]:
df_new_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99000107 entries, 0 to 99000106
Data columns (total 5 columns):
userId          int64
pageId          int64
visit_number    int64
topicId         int64
confidence      float64
dtypes: float64(1), int64(4)
memory usage: 4.4 GB


In [8]:
df_new_2['VC'] = df_new_2['visit_number'] * df_new_2['confidence']

In [9]:
user_feature = df_new_2.groupby(['userId','topicId']).\
                                        agg({'VC':'sum'}).reset_index()

In [10]:
user_feature.sample(10)

,userId,topicId,VC
13667967,1914601,42,3.349052
2274605,295767,48,1.279326
15100750,2130264,47,2.430341
779116,110025,30,0.025878
6890716,883095,31,0.195412
6768514,865869,32,0.205327
7239775,936372,35,1.371908
1051317,140572,3,0.560390
1870516,238145,45,1.027537
2324531,302679,31,0.178736


In [13]:
user_feature['VC'].var()

292.89306339393767

In [12]:
user_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361114 entries, 0 to 18361113
Data columns (total 3 columns):
userId     int64
topicId    int64
VC         float64
dtypes: float64(1), int64(2)
memory usage: 420.3 MB


In [14]:
user_feature.to_csv('user_feature.csv')